# Bayesian Search Algorithm

Use bayesian search based on the paper: "Bayesian-Based Search Decision Framework and Search Strategy Analysis in Probabilistic Search"

link: https://onlinelibrary.wiley.com/doi/full/10.1155/2020/8865381

In [111]:
# Import necessary libraries
import networkx as nx
import random
import re
import xml.etree.ElementTree as ET

In [112]:
# Constants
map_graph_path = "../graphs/original/final_graph.graphml" # .graphml file path
start_node = "n0"   # Start position (node)
target_node = "n20" # target's (ring) position (node)
suspected_nodes = ["n20", "n25", "n22"]
p_tp = 0.8  # P(true positive)
p_fp = 0.2  # P(false positive)
p_tn = 0.9 # P(true negative)
p_fn = 0.1  # P(false negative)
B_lower = 0  # Lower boundary for maximum belief
B_upper = 1   # Upper boundary for maximum belief
max_steps = 500  # Maximum number of steps

# May I say "Fuck this paper's notations. What the fuck do you mean by 3 same equations having 3 distinct values??? What the hell is this level of inconsistency in naming variables???."

In [113]:
# Generic helper functions

# Display graph information
def display_graph_information(graph):
    # Print nodes data
    for node, data in graph.nodes(data=True):
        print(f"Node: {node}, Attributes: {data}")

    # Print edges data
    for u, v, data in graph.edges(data=True):
        print(f"Edge: ({u}, {v}), Attributes: {data}")

    print("Number of Nodes:", len(graph.nodes()))
    print("Number of Edges:", len(graph.edges()))

def add_edges_weight(graph):
    for u, v, data in graph.edges(data=True):
        data['weight'] = 1.0  # Assign a weight of 1 to each edge

In [114]:
# Set target (ring) position (node)
def set_target_node(graph, target_node_id):
    nx.set_node_attributes(graph, {target_node_id: {"target": True}})

In [115]:
# Bayesian search for incomplete information sensor

# Create an initial belief map
def initialize_belief_map(graph, choice, suspected_nodes=[]):
    """
        Choice:
            0: Uniform Belief Map
            1: Randomized Belief Map
            2: Peak (require suspected_node)
            3: Normal Distribution Map (require suspected_node)
    """
    num_nodes = len(graph.nodes())

    if choice == 0:
        uniform_belief = 1.0 / num_nodes

        for node in graph.nodes():
            graph.nodes[node]["P(target)"] = uniform_belief
    elif choice == 1:
        for node in graph.nodes():
            graph.nodes[node]["P(target)"] = random.uniform(0.0, 1.0)
    elif choice == 2 and len(suspected_nodes) > 0:
        suspected_belief = 1.0 / len(suspected_nodes)
        general_belief = 1.0 / (num_nodes - len(suspected_nodes))
        
        for node in graph.nodes():
            graph.nodes[node]["P(target)"] = general_belief
        for suspected_node in suspected_nodes:
            graph.nodes[suspected_node]["P(target)"] = suspected_belief
        
    elif choice == 3 and len(suspected_nodes) > 0:
        sd = 1 
        mean = 0.5


# Implement the Dijkstra algorithm
def dijkstra_search(graph, start_node, target_node):
    try:
        path = nx.dijkstra_path(graph, source=start_node, target=target_node, weight='weight')
        
        return path
    except nx.NetworkXNoPath:
        print(f"No path found between {start_node} and {target_node}.")
        return []

# Update the belief map
def update_belief_map(graph, agent_node, observation):
    total_nodes = len(graph.nodes())
    new_beliefs = {}
    
    # Calculate the denominator for the Bayesian update (marginal probability)
    denominator = 0.0

    for node in graph.nodes():
        prior_prob = graph.nodes[node]["P(target)"]

        if node == agent_node:
            if observation:
                denominator += prior_prob * p_tp  # P(O | D_d) * P(D_d)
            else:
                denominator += prior_prob * p_fn  # P(~O | D_d) * P(D_d)
        else:
            if observation:
                denominator += prior_prob * p_fp  # P(O | D_c) * P(D_c)
            else:
                denominator += prior_prob * p_tn  # P(~O | D_c) * P(D_c)

    # Quit if denominator = 0
    if denominator == 0:
        return

    # Updated probabilities
    new_beliefs = {}

    for node in graph.nodes():
        prior_prob = graph.nodes[node]["P(target)"]

        if node == agent_node:
            if observation:
                numerator = prior_prob * p_tp  # P(D_d | O)
            else:
                numerator = prior_prob * p_fn  # P(D_d | ~O)
        else:
            if observation:
                numerator = prior_prob * p_fp  # P(D_c | O)
            else:
                numerator = prior_prob * p_tn  # P(D_c | ~O)
                
        updated_prob = numerator / denominator
        new_beliefs[node] = updated_prob

    # Assign the new beliefs to the graph nodes
    nx.set_node_attributes(graph, new_beliefs, "P(target)")

# Modified saccadic search strategy
def saccadic_search_strategy(graph, start_node, max_steps, target_node, suspected_nodes=[]):
    current_node = start_node
    path_taken = [current_node]
    target_found = False
    count = 1
    
    # Initialize with uniform belief
    if len(suspected_nodes) == 0:
        initialize_belief_map(graph, choice=0)
    elif len(suspected_nodes) > 0:
        initialize_belief_map(graph, choice=2, suspected_nodes=suspected_nodes)

    current_path = []
    
    for step in range(max_steps):
        # print(f"Step: {step + 1}")
        
        # Check if we have reached our planned destination and need a new one
        if not current_path:
            # Find the node with the highest belief
            highest_belief_node = max(graph.nodes(data='P(target)'), key=lambda x: x[1])[0]
            
            # print(f"New target destination identified: {highest_belief_node}.")
            
            # Plan a new path to the highest belief node
            current_path = dijkstra_search(graph, current_node, highest_belief_node)
            
            # If no path is found, break the loop
            if not current_path:
                # print(f"No path found to {highest_belief_node}.")
                
                break
                
            # The first element is the current node, we want to move to the next.
            if len(current_path) > 1:
                current_path.pop(0)

        # Move to the next node on the path
        if current_path:
            next_node = current_path.pop(0)

            # print(f"Moving from {current_node} to {next_node}")

            current_node = next_node

            path_taken.append(current_node)
        
        # Perform observation and update belief at the new location
        is_target = graph.nodes[current_node].get("target", False)
        
        if is_target:
            observation = random.random() < p_tp   # True positive with probability p_tp
        else:
            observation = random.random() < p_fp   # False positive with probability p_fp
        
        update_belief_map(graph, current_node, observation)
        # print(f"At {current_node}. Belief at this node is {graph.nodes[current_node]['P(target)']:.4f}")

        # Check for target presence. We check at every step because we're moving and observing.
        if current_node == target_node:
            #  print(f"Target found at {current_node}!")

             target_found = True

             break
        
        count+=1
    
    print("Number of steps taken: ", count)
    
    return path_taken, target_found

In [116]:
# Read .graphml file
map_graph = nx.read_graphml(map_graph_path)

display_graph_information(map_graph)

Node: n0, Attributes: {'x': '539.7320177831189', 'y': '814.3710124080101', 'shape_type': 'ellipse', 'label': '16,11'}
Node: n1, Attributes: {'x': '491.39296206082173', 'y': '814.701509196422', 'shape_type': 'ellipse', 'label': '16,10'}
Node: n2, Attributes: {'x': '540.242019608562', 'y': '765.0825618080634', 'shape_type': 'ellipse', 'label': '15,11'}
Node: n3, Attributes: {'x': '589.4380120008074', 'y': '813.8610105825668', 'shape_type': 'ellipse', 'label': '16,12'}
Node: n4, Attributes: {'x': '491.50793350674974', 'y': '764.5725599826202', 'shape_type': 'ellipse', 'label': '15,10'}
Node: n5, Attributes: {'x': '539.5620171746378', 'y': '862.7626250900943', 'shape_type': 'ellipse', 'label': '17,11'}
Node: n6, Attributes: {'x': '490.88296023537856', 'y': '862.2526232646511', 'shape_type': 'ellipse', 'label': '17,10'}
Node: n7, Attributes: {'x': '590.2080004396669', 'y': '764.5725599826202', 'shape_type': 'ellipse', 'label': '15,12'}
Node: n8, Attributes: {'x': '589.0179961802995', 'y': '

In [117]:
# Run the search and get results
print("\nStarting Saccadic Search...")

search_path, found = saccadic_search_strategy(map_graph, start_node, max_steps, target_node, suspected_nodes=suspected_nodes)

print("\nSearch Results:")  

if found:
    print("Target was found!")
else:
    print("Target was not found within the maximum number of steps.")
    
print("Search path:", search_path)


Starting Saccadic Search...
Number of steps taken:  3

Search Results:
Target was found!
Search path: ['n0', 'n2', 'n10', 'n20']


In [118]:
for searched_node in search_path:
    for node, data in map_graph.nodes(data=True):
        if searched_node == node:
            print(f"Node: {node}, Attributes: {data}")
            print("X:", int(float(data["x"]) / 50))
            print("Y:", int(float(data["y"]) / 50))

Node: n0, Attributes: {'x': '539.7320177831189', 'y': '814.3710124080101', 'shape_type': 'ellipse', 'label': '16,11', 'P(target)': 0.0008209681342738865}
X: 10
Y: 16
Node: n2, Attributes: {'x': '540.242019608562', 'y': '765.0825618080634', 'shape_type': 'ellipse', 'label': '15,11', 'P(target)': 9.12186815859874e-05}
X: 10
Y: 15
Node: n10, Attributes: {'x': '589.6080126092883', 'y': '716.3325161719835', 'shape_type': 'ellipse', 'label': '14,12', 'P(target)': 9.121868158598739e-05}
X: 11
Y: 14
Node: n20, Attributes: {'x': '636.9970729225594', 'y': '717.6686147042643', 'shape_type': 'ellipse', 'label': '14,13', 'P(target)': 0.02177085867185566}
X: 12
Y: 14


## Utils

In [119]:
LABEL_RE = re.compile(r"^\s*-?\d+\s*,\s*-?\d+\s*$")
NS = {
    "g": "http://graphml.graphdrawing.org/xmlns",
    "y": "http://www.yworks.com/xml/graphml",
}
map_graph_path = "../mapGraphs/graphml/HIMCM_graph_FINAL.graphml" # .graphml file path

def inject_node_labels_from_graphml_file(G, graphml_path, label_attr="node_label"):
    tree = ET.parse(str(graphml_path))
    root = tree.getroot()
 
    for n in root.findall(".//g:graph/g:node", NS):
        nid = n.get("id")
        if nid not in G:  # only inject for nodes that exist in this graph
            continue
 
        txt = None
        for d in n.findall("./g:data", NS):
            lab = d.find("./y:ShapeNode/y:NodeLabel", NS)
            if lab is None:
                lab = d.find("./y:GenericNode/y:NodeLabel", NS)
            if lab is not None and lab.text:
                txt = lab.text.strip()
                break
 
        # If it's a coordinate-like label, use it; otherwise keep any existing value
        if txt and LABEL_RE.match(txt):
            G.nodes[nid][label_attr] = txt
 
def nodes_to_labels(G, node_id_path, label_attr="node_label"):
    return [G.nodes[nid].get(label_attr, str(nid)) for nid in node_id_path]

## Test Cases

### Static Test

In [120]:
start_nodes = ["n0", "n20", "n33", "n200", "n555"]
target_nodes = ["n20", "n22", "n576", "n714", "n123"]
max_steps = 15000

In [121]:
def static_test(graph, start_nodes, target_nodes):
    for start_node, target_node in zip(start_nodes, target_nodes):
        print("Start Node: " + start_node, "Target Node: " + target_node)

        search_path, found = saccadic_search_strategy(graph, start_node, max_steps, target_node)
        
        inject_node_labels_from_graphml_file(graph, map_graph_path, label_attr="y:LabelModel")

        path_labels = nodes_to_labels(graph, search_path, label_attr="y:LabelModel")
        
        print("Path (labels):", " -> ".join(path_labels))
        print("Target found:", found)

In [122]:
map_graph = nx.read_graphml(map_graph_path)

static_test(map_graph, start_nodes, target_nodes)


Start Node: n0 Target Node: n20
Number of steps taken:  52
Path (labels): 16,11 -> 16,11 -> 16,10 -> 15,11 -> 16,12 -> 16,12 -> 16,12 -> 16,12 -> 16,12 -> 16,11 -> 15,10 -> 16,10 -> 17,11 -> 17,10 -> 17,10 -> 17,10 -> 17,10 -> 16,11 -> 15,12 -> 16,12 -> 17,12 -> 16,11 -> 15,11 -> 14,11 -> 14,12 -> 15,11 -> 14,10 -> 14,9 -> 15,9 -> 16,9 -> 17,10 -> 18,11 -> 18,11 -> 18,11 -> 18,11 -> 17,12 -> 16,13 -> 16,12 -> 16,11 -> 17,10 -> 17,9 -> 17,9 -> 17,10 -> 16,11 -> 15,11 -> 14,12 -> 15,13 -> 15,13 -> 16,13 -> 17,13 -> 16,13 -> 15,14 -> 14,13
Target found: True
Start Node: n20 Target Node: n22
Number of steps taken:  50
Path (labels): 14,13 -> 14,12 -> 15,11 -> 16,11 -> 16,10 -> 16,11 -> 16,12 -> 16,11 -> 15,10 -> 16,10 -> 17,11 -> 17,11 -> 17,10 -> 16,11 -> 15,12 -> 16,12 -> 17,12 -> 16,11 -> 15,11 -> 14,11 -> 14,10 -> 14,9 -> 15,9 -> 16,9 -> 17,10 -> 18,11 -> 17,12 -> 16,13 -> 16,12 -> 16,11 -> 17,10 -> 17,9 -> 17,9 -> 17,10 -> 16,11 -> 15,11 -> 14,12 -> 15,13 -> 16,13 -> 17,13 -> 16,13 ->

### Random Test

In [123]:
number_of_test_cases = 5
max_steps = 15000

In [124]:
def random_test(graph, number_of_test_cases):
    number_of_nodes = len(graph.nodes())
    start_nodes = []
    target_nodes = []

    for i in range(number_of_test_cases):
        start_nodes.append("n" + str(random.randint(0, number_of_nodes)))
        target_nodes.append(("n" + str(random.randint(0, number_of_nodes))))
    
    static_test(graph, start_nodes, target_nodes)

In [125]:
map_graph = nx.read_graphml(map_graph_path)

random_test(map_graph, number_of_test_cases)

Start Node: n148 Target Node: n349
Number of steps taken:  963
Path (labels): 13,4 -> 12,5 -> 13,6 -> 13,7 -> 13,8 -> 14,9 -> 15,10 -> 16,11 -> 16,10 -> 16,10 -> 15,11 -> 16,12 -> 17,11 -> 17,10 -> 16,11 -> 15,12 -> 16,12 -> 17,12 -> 16,11 -> 15,11 -> 14,11 -> 14,12 -> 15,11 -> 14,10 -> 14,10 -> 15,9 -> 16,9 -> 17,10 -> 18,11 -> 18,11 -> 17,12 -> 16,13 -> 16,12 -> 16,11 -> 17,10 -> 17,9 -> 17,10 -> 16,11 -> 15,11 -> 14,12 -> 15,13 -> 16,13 -> 17,13 -> 16,13 -> 15,14 -> 14,13 -> 14,13 -> 14,12 -> 15,11 -> 16,11 -> 17,10 -> 18,10 -> 17,11 -> 18,12 -> 17,11 -> 18,10 -> 18,9 -> 18,10 -> 17,11 -> 17,12 -> 18,13 -> 17,12 -> 17,11 -> 18,10 -> 18,9 -> 17,8 -> 16,7 -> 16,6 -> 16,5 -> 15,6 -> 15,5 -> 15,5 -> 15,5 -> 15,5 -> 15,5 -> 16,6 -> 17,6 -> 17,5 -> 16,6 -> 15,7 -> 16,8 -> 17,7 -> 16,6 -> 15,6 -> 14,6 -> 14,7 -> 14,6 -> 14,5 -> 14,4 -> 15,4 -> 16,4 -> 17,5 -> 18,6 -> 18,6 -> 18,5 -> 17,4 -> 18,5 -> 18,6 -> 17,7 -> 16,8 -> 15,8 -> 14,8 -> 14,8 -> 15,8 -> 16,8 -> 17,7 -> 18,7 -> 17,7 -> 17,6